## 调用方式
1. 流式调用、非流式调用
2. 批次调用、非批次调用
3. 同步调用、异步调用

### 获取模型实例

In [1]:
from langchain.chat_models import init_chat_model
from langchain.messages import SystemMessage, HumanMessage

# 1. 获取大模型实例
model = init_chat_model(model="deepseek-chat", model_provider="deepseek")

# 2. 构造一个提示词模版或提示词
messages = [
    SystemMessage(content="你是一个智能助手 ，帮用户解决问题，你的名字叫一把手"),
    HumanMessage(content="你是谁？"),
]

#### 非流式调用

In [3]:
model.invoke(messages)

AIMessage(content='你好！我是“一把手”，一个智能助手，专门为你提供帮助、解答问题或协助处理各种任务。有什么我可以帮你的吗？', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 29, 'prompt_tokens': 19, 'total_tokens': 48, 'completion_tokens_details': None, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}, 'prompt_cache_hit_tokens': 0, 'prompt_cache_miss_tokens': 19}, 'model_provider': 'deepseek', 'model_name': 'deepseek-chat', 'system_fingerprint': 'fp_eaab8d114b_prod0820_fp8_kvcache', 'id': '959bbbc6-05e5-4b2e-8c5b-924cb721ed43', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--de1ab2ec-1e5f-490f-b60a-6d792c83b21d-0', usage_metadata={'input_tokens': 19, 'output_tokens': 29, 'total_tokens': 48, 'input_token_details': {'cache_read': 0}, 'output_token_details': {}})

#### 流式调用

In [4]:
# res为生成器
res = model.stream("你是谁？")
print(f"res: {res}")

for chunk in res:
    print(chunk.content, end="")

res: <generator object BaseChatModel.stream at 0x115560220>
你好！我是DeepSeek，由深度求索公司创造的AI助手！😊

我是一个纯文本模型，虽然不支持多模态识别功能，但我有文件上传功能，可以帮你处理图像、txt、pdf、ppt、word、excel等各种文件，从中读取文字信息进行分析处理。

我的一些特点：
- 完全免费使用，没有收费计划
- 上下文长度达到128K，可以处理很长的对话
- 支持联网搜索（需要你在Web/App中手动点开联网搜索按键）
- 可以通过官方应用商店下载App使用

我的知识截止到2024年7月，会以热情细腻的方式为你提供帮助。无论是回答问题、协助思考、处理文档，还是进行创意写作，我都很乐意帮忙！

有什么我可以为你做的吗？✨

#### 批次调用
底层是通过多线程的方式去调用

In [6]:
messages = [
    [
        {"role": "system", "content": "你是以为诗人。"},
        {"role": "user", "content": "写一句关于春天的诗。"},
    ],
    [
        {"role": "system", "content": "你是以为诗人。"},
        {"role": "user", "content": "写一句关于夏天的诗。"},
    ],
    [
        {"role": "system", "content": "你是以为诗人。"},
        {"role": "user", "content": "写一句关于秋天的诗。"},
    ],
]

res = model.batch(messages)
print(f"res[0]: {res[0].content}")
print(f"res[1]: {res[1].content}")
print(f"res[2]: {res[2].content}")

res[0]: 风扶弱柳试新弦，一针绿雨绣江南。
res[1]: 《夏午》
一蝉分竹色，半榻借溪光。
忽有微凉起，风扶荷叶香。
res[2]: 霜叶题红笺，风诵满山诗。


#### 同步调用
多次请求串行处理

In [7]:
import time

messages = [
    [
        {"role": "system", "content": "你是以为诗人。"},
        {"role": "user", "content": "写一句关于春天的诗。"},
    ],
    [
        {"role": "system", "content": "你是以为诗人。"},
        {"role": "user", "content": "写一句关于夏天的诗。"},
    ],
    [
        {"role": "system", "content": "你是以为诗人。"},
        {"role": "user", "content": "写一句关于秋天的诗。"},
    ],
]

start_time = time.time()
res = [model.invoke(message) for message in messages]
end_time = time.time()
print(f"总耗时：{end_time - start_time}")

总耗时：5.462079048156738


#### 异步调用
协程是单线程，是实现的一步调用，并发能力是高于多线程的。能够减少资源（线程数）的使用量

In [ ]:
import asyncio

messages = [
    [
        {"role": "system", "content": "你是以为诗人。"},
        {"role": "user", "content": "写一句关于春天的诗。"},
    ],
    [
        {"role": "system", "content": "你是以为诗人。"},
        {"role": "user", "content": "写一句关于夏天的诗。"},
    ],
    [
        {"role": "system", "content": "你是以为诗人。"},
        {"role": "user", "content": "写一句关于秋天的诗。"},
    ],
]


async def gather_task(messages: list):
    tasks = [model.ainvoke(message) for message in messages]
    print(f"tasks: {tasks}")
    res = await asyncio.gather(*tasks)
    print(f"res: {res}")


await gather_task(messages)
# res = await gather_task(messages)
# print(f"res: {res}")

tasks: [<coroutine object BaseChatModel.ainvoke at 0x1151f9df0>, <coroutine object BaseChatModel.ainvoke at 0x1151f9b90>, <coroutine object BaseChatModel.ainvoke at 0x1151f9cc0>]
res: [AIMessage(content='风扶弱柳试新弦，细雨斜织绿野烟。\n忽有早莺争暖树，一啼惊破杏花天。', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 31, 'prompt_tokens': 14, 'total_tokens': 45, 'completion_tokens_details': None, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}, 'prompt_cache_hit_tokens': 0, 'prompt_cache_miss_tokens': 14}, 'model_provider': 'deepseek', 'model_name': 'deepseek-chat', 'system_fingerprint': 'fp_eaab8d114b_prod0820_fp8_kvcache', 'id': '8f0885e9-4dd9-45b7-9bc8-df6d69c628c5', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--8c75bd35-e0fb-49a5-a198-2b8e7c581eec-0', usage_metadata={'input_tokens': 14, 'output_tokens': 31, 'total_tokens': 45, 'input_token_details': {'cache_read': 0}, 'output_token_details': {}}), AIMessage(content='《夏午》\n一蝉衔走整座山的寂静\n光在叶脉